# Introduction

In this notebook we will learn how to **characterize bus repairs** from free text descriptions, entered by users. This will be accomplished through the use of **Natural Language Processing (NLP)**.

This will allow you to discover, step by step, how you can create the code doing the repair text processing.  In the last part of the workshop, this code will be **packaged to create a service** that you can query from an application.

We will be training the model.  Once our model is trained, we can test the model by entering a bus repair issue (e.g. the brakes feel soft when I press on them) and check if the model has correctly characterized the claim.  Repairs will be categorized as:  Brakes, Starter or Other.

Ready? Let's go!

# Environment initialization

## Libraries
First, we'll need to **install some libraries** that are not part of our container image. Normally, **Red Hat OpenShift Data Science** is already taking care of this for you, based on what it detects in the code. **Red Hat OpenShift Data Science** will reinstall all those libraries for you every time you launch the notebook!


## Imports
Of course, we'll need to import various packages. They are either built in the notebook image you are running, or have been installed in the previous step.

In [ ]:
import numpy as np 
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow import keras


## Create Training and Testing data sets

Now that we have loaded the tools we need, the first step in our journey is to be able to take our raw data and divide it into testing and training sets.


In [ ]:
#============================================================================
#Determine what the training and testing percentages will be.
#============================================================================

training_portion = .80  # Use 80% of data for training, 20% for testing
max_words = 1000        #Max words in text input

data = pd.read_csv('testdata1.csv')

#print out first 5 rows of generated claims so you can see what data looks like
print(data.head())

train_size = int(len(data) * training_portion)

#============================================================================
# FUNCTION:  train_test_split - splits the data into training and test sets.  
# Inputs are the raw data and determined train_size.
#============================================================================
def train_test_split(data, train_size):
    train = data[:train_size]
    test = data[train_size:]
    return train, test

train_cat, test_cat = train_test_split(data.iloc[:,1], train_size)  # label data is second column
train_text, test_text = train_test_split(data.iloc[:,0], train_size)  # text data is first column

## Tokenize the Data sets

After we have training and testing sets, we need to **tokenize the data**.  This means that we convert text documents into contextual vectors which contain numeric representations (index of where those words occur in a word dictionary) of the words in the documents.

To see how Tokenization works, you can take a look at **02-TokenDemo.ipynb**.

In [ ]:
tokenize = Tokenizer(num_words=max_words, char_level=False)
tokenize.fit_on_texts(train_text) # fit tokenizer to our training text data

#============================================================================
#x_train and x_test are the vectorization of the text data (which is a claim)
#============================================================================
x_train = tokenize.texts_to_matrix(train_text)

#following print statement shows some rows in the newly created matrix
print(x_train)
x_test = tokenize.texts_to_matrix(test_text)

In [3]:
#============================================================================
# Use sklearn utility to convert label strings to numbered index
#============================================================================

encoder = LabelEncoder()  
encoder.fit(train_cat)

#convert label strings to numbers
y_train = encoder.transform(train_cat)
y_test = encoder.transform(test_cat)

#============================================================================
#for each row in the data, each entry represents the value of the label
#============================================================================
print(y_train)

#============================================================================
# for example:  [2 1 1 2 1 1 0 ...  which corresponds to starter, other,
# other, starter, other, other, brakes ...
#============================================================================

[2 1 2 1 1 0 0 0 1 1 0 1 0 2 1 2 1 1 0 2 1 2 0 1 1 1 1 0 2 1 0 2 2 1 0 2 1
 1 1 0 1 0 1 0 0 1 1 0 0 1 0 0 1 0 0 1 1 0 0 0 1 2 1 2 1 0 0 1 1 1 1 1 1 2
 0 1 0 1 1 2 1 0 0 0 0 1 2 1 1 1 1 2 2 1 2 0 1 0 2 1 2 1 1 2 1 1 1 2 0 1 1
 1 0 0 1 0 1 1 2 0 0 1 2 0 1 1 0 1 1 1 2 0 0 0 2 2 0 0 2 2 1 1 0 0 2 1 0 0
 0 1 2 2 0 2 0 1 2 2 2 2 1 1 0 1 1 2 0 1 2 0 1 2 1 1 0 0 1 1 1 2 1 1 0 1 0
 2 2 0 2 1 2 1 2 0 1 1 0 0 1 1 2 1 1 0 0 1 2 2 1 1 1 2 0 2 2 1 1 2 1 0 1 2
 1 0 2 1 1 0 1 1 1 2 1 0 1 1 1 2 2 2 1 1 1 1 1 2 0 0 2 0 1 2 2 2 1 1 2 2 1
 1 1 2 2 1 2 2 1 1 1 1 1 2 2 1 1 1 0 0 1 1 0 1 0 1 1 0 2 1 2 2 1 1 2 2 1 1
 0 1 1 2 1 1 2 2 0 0 0 2 0 1 1 1 1 2 1 1 1 0 1 0 0 2 1 2 2 2 0 2 0 1 0 0 1
 1 1 0 2 0 0 1 2 1 2 1 1 1 2 1 1 0 1 1 0 0 2 2 1 1 1 2 1 0 1 2 0 1 0 1 1 2
 2 1 1 1 0 1 0 1 1 1 0 0 0 1 1 0 1 1 1 2 1 2 2 1 0 1 1 1 1 1 2 1 1 0 2 1 1
 1 1 1 1 0 1 2 0 0 1 2 1 1 1 1 1 1 1 2 0 0 2 2 0 0 1 2 0 1 1 0 2 2 1 0 2 1
 2 1 0 1 0 1 1 0 2 0 1 2 2 1 0 1 1 0 1 2 1 0 0 1 2 1 2 0 2 1 1 1 1 2 1 0 1
 1 1 1 1 1 1 1 0 1 2 1 2 

## One Hot Encoding

We need to create labels (categories such as Brakes or Starter) for our test data, convert the labels to numbered index and then use one-hot encoding.

**One hot encoding** allows the representation of categorical data to be more expressive. Many machine learning algorithms cannot work with categorical data directly. **The categories must be converted into numbers**. This is required for both input and output variables that are categorical.

After we have converted the labels using one-hot encoding, we will be ready to build our main NLP model and train it.

In [4]:
#============================================================================
# Convert the labels to a one-hot representation
#============================================================================

num_classes = len(set(y_train))  # set() creates a unique set of objects
y_train = to_categorical(y_train, num_classes)  

#============================================================================
# one hot encoding replaces the column of labels whose (values are 0 or 1 or 2)
# with 3 columns each representing 1 label value.  For example, the label 
# 'other' is replaced by the vector 0 1 0, the label 'starter' is replaced by
# the vector 0 0 1, the label 'brakes' is replaced by the vector 1 0 0
#============================================================================

y_test = to_categorical(y_test, num_classes)

#============================================================================
# Inspect the dimenstions of our training and test data
#============================================================================
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

#one hot encoding
#y_train shape: (159, 3)  159 rows, 3 columns
#y_test shape: (40, 3)  40 rows, 3 columns

x_train shape: (799, 1000)
x_test shape: (200, 1000)
y_train shape: (799, 3)
y_test shape: (200, 3)



## Building the model

Once the model is trained, we can test our model by entering a repair issue (e.g. the brakes feel soft when I press on them) and check if the model has correctly characterized the repair issue. 


In [5]:
#============================================================================
# Build the model
#============================================================================
layers = keras.layers
models = keras.models
model = models.Sequential()
model.add(layers.Dense(512, input_shape=(max_words,), activation='relu'))  # Hidden layer with 512 nodes
model.add(layers.Dense(num_classes, activation='softmax'))

#============================================================================
# relu, softmax, categorical_crossentropy are telling the model how to do some 
# internal calculations.  Softmax is telling the model to calculate 
# probabilities for each category in each document.  If you only had yes, 
# or no you would use sigmoid instead of softmax.
#============================================================================
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

#============================================================================
# The variable, history, is normally used to plot learning curves.  fit 
# calculates the weights in the model.  
# batch_size tells the internal calculations how many rows to process at one time
# epochs is the number of times the model calculations will pass through 
# the entire data
#============================================================================
batch_size = 32
epochs = 2
history = model.fit(x_train, y_train,      
                    batch_size=batch_size,  
                    epochs=epochs,         
                    verbose=1,
                    validation_split=0.1)

#============================================================================
#evaluate compares the model predictions with the actual known test values
#============================================================================
score = model.evaluate(x_test, y_test,       
                       batch_size=batch_size, verbose=1)

#============================================================================
#print the test loss and accuracy of our model
#============================================================================
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Epoch 1/2
23/23 [==============================] - 1s 27ms/step - loss: 0.9808 - accuracy: 0.4801 - val_loss: 0.5619 - val_accuracy: 0.9875
Epoch 2/2
7/7 [==============================] - 0s 1ms/step - loss: 0.2539 - accuracy: 0.9550
Test loss: 0.25385844707489014
Test accuracy: 0.9549999833106995



## Let's test our model!

Now that we have a model, we would like to generate a prediction (e.g. categorize the repair issue as:  Brakes, Starter or Other)


In [6]:
# Here's how to generate a prediction on individual examples
text_labels = encoder.classes_   #ndarray of output values (labels or classes)  e.g. other, brakes, starter

# Examine first 10 test samples of 445
for i in range(len(test_cat)):
    temp = x_test[i]
    prediction = model.predict(np.array([x_test[i]]))
    predicted_label = text_labels[np.argmax(prediction)]  #predicted class
    #print(test_text.iloc[i][:50], "...")                # 50 char sample of text
    #print('Actual label:' + test_cat.iloc[i])
    #print("Predicted label: " + predicted_label + "\n")

#test our model by inputing a new user claim =============   
#print('================================  ENTER TEST PHRASE ====================================')
#single_test_text = 'press brake pedal and car wont stop'  #text has to be a document, the brackets make it a document
single_test_text = 'turn key over and hear a clicking sound' 
#single_test_text = 'there is fluid leaking from the engine' 

#print("Entered text: " + str(single_test_text) + "\n")

#text_as_nparray = np.array([single_test_text])
text_as_series = pd.Series(single_test_text)  #data conversions

single_x_test = tokenize.texts_to_matrix(text_as_series)
single_prediction = model.predict(np.array([single_x_test]))
single_predicted_label = text_labels[np.argmax(single_prediction)]  #maps index of the prediction to the test labels array e.g. brakes
#print("Predicted label: " + single_predicted_label + "\n")

#======================================
#test predication function
#======================================
#print('=====================test prediction function==================================')

def predict(single_test_text):
    text_as_series = pd.Series(single_test_text) #do a data conversion
    single_x_test = tokenize.texts_to_matrix(text_as_series)
    single_prediction = model.predict(np.array([single_x_test]))
    single_predicted_label = text_labels[np.argmax(single_prediction)]  #maps index of the prediction to the test labels array e.g. brakes
    return (single_predicted_label)

prediction = predict(single_test_text)  #call the prediction function

print('returned prediction: ' + prediction)

returned prediction: starter


In [7]:
prediction = predict('there is a pool of liquid under the car')
print(prediction)

other
